# Spatial analysis of simulation results

In [21]:
import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import ipywidgets as widgets
from natsort import natsorted
import os
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors

In [2]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 120

In [9]:
# Config

gcms = ['canesm2_RCP85', 'ccsm4_RCP85', 'giss_e2_h_RCP85', 'noresm1_m_RCP85']
sim_start = pd.to_datetime('01-01-2020')
sim_end = pd.to_datetime('12-31-2099')

In [10]:
# Import results files

scenarios = ['ind35yr']

dailies = []
annuals = []

# Import daily and annual results
# Results in nested lists ([x][y], for x management scenarios and y GCMs)
for scenario in scenarios:
    dailies_scenario = []
    annuals_scenario = []
    scenario_dir = config.velma_data.parents[1] / 'results' / scenario
    dirs = os.listdir(scenario_dir)
    for gcm in gcms:
        results_dir = scenario_dir / 'ellsworth_{}_{}_{}_{}'.format(scenario,
                                                                    sim_start.year % 100,
                                                                    sim_end.year % 100,
                                                                    gcm)

        daily_results = pd.read_csv(results_dir / 'DailyResults.csv')

        # Format datetime
        jday_pad = daily_results['Day'].apply(lambda x: str(x).zfill(3))
        str_year = daily_results['Year'].apply(lambda x: str(x))
        rng = pd.to_datetime((str_year + jday_pad), format='%Y%j')
        daily_results.index = rng
        dailies_scenario.append(daily_results)

    dailies.append(dailies_scenario)

## Biomass carbon

In [50]:
# Trim the image to only the delineated watershed simulated through VELMA
# Can export a delineated DEM from JPDEM
del_dem = np.loadtxt(config.dem_velma.parents[0] / 'delineated_dem.asc', skiprows=6)
watershed_mask = (del_dem==-9999)

biomass_c_dir = results_dir / 'spatial_writer' / 'biomass_c'

biomass_c_asc = []
for file in os.listdir(biomass_c_dir):
    asc = np.loadtxt(biomass_c_dir / file, skiprows=6)
    asc[watershed_mask] = np.nan
    biomass_c_asc.append(asc)

sim_years = []
for file in os.listdir(biomass_c_dir):
    year = Path(file).stem.split('_')[-2]
    sim_years.append(year)
years = [int(x) for x in sim_years]

In [68]:
results_dir

WindowsPath('C:/Users/ipdavies/tnc_velma/notebooks/../results/ind35yr/ellsworth_ind35yr_20_99_noresm1_m_RCP85')

In [67]:
binary_cmap = cm.get_cmap('binary', 2)
gray = np.array([0.31, 0.31, 0.31, 1])
binary_cmap_colors = binary_cmap(np.linspace(0, 1, 2))
binary_cmap_colors[:1, :] = gray
binary_cmap_gray = colors.ListedColormap(binary_cmap_colors)

old_cmap = cm.get_cmap('YlGn', 256)
newcolors = old_cmap(np.linspace(0, 1, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
newcolors[:2, :] = pink
new_cmap = colors.ListedColormap(newcolors)

@widgets.interact(year=(2020, 2099))
def f(year=min(sim_years)):
    ind = sim_years.index(str(year))
    fig, ax = plt.subplots(figsize=(5, 6))
    im = ax.imshow(biomass_c_asc[ind], cmap=new_cmap)
    mask = np.isnan(biomass_c_asc[ind]) * 1.0
    mask[mask == 0] = np.nan
    ax.imshow(mask, cmap=binary_cmap_gray)
    cbar = fig.colorbar(im, ax=ax)
    cbar.ax.set_ylabel(r'$gC/m^2$', rotation=270)

interactive(children=(IntSlider(value=2020, description='year', max=2099, min=2020), Output()), _dom_classes=(…

## Filter maps

In [43]:
# Trim the image to only the delineated watershed simulated through VELMA
# Can export a delineated DEM from JPDEM
del_dem = np.loadtxt(config.dem_velma.parents[0] / 'delineated_dem.asc', skiprows=6)
watershed_mask = (del_dem==-9999)

filter_maps_dir = config.velma_data / 'landcover' / 'filter_maps'

asciis = []
name = 'random_35yr_clearcut_10pct'
for file in natsorted(os.listdir(filter_maps_dir)):
    if name in file:
        z = np.loadtxt(filter_maps_dir / file, skiprows=6)
        z[watershed_mask] = np.nan
        asciis.append(z)

In [47]:
binary_cmap = cm.get_cmap('binary', 2)
gray = np.array([0.31, 0.31, 0.31, 1])
binary_cmap_colors = binary_cmap(np.linspace(0, 1, 2))
binary_cmap_colors[:1, :] = gray
binary_cmap_gray = colors.ListedColormap(binary_cmap_colors)

years = [x+1 for x in range(0, len(asciis))]
@widgets.interact(year=(np.min(years), np.max(years)))
def f(year=min(years)):
    ind = years.index(year)
    fig, ax = plt.subplots(figsize=(5, 6))
    im = ax.imshow(asciis[ind], cmap='Reds')
    mask = np.isnan(asciis[ind]) * 1.0
    mask[mask == 0] = np.nan
    ax.imshow(mask, cmap=binary_cmap_gray)
    cbar = fig.colorbar(im, ax=ax)
    cbar.ax.set_ylabel('Harvested/Not harvested', rotation=270)

interactive(children=(IntSlider(value=1, description='year', max=13, min=1), Output()), _dom_classes=('widget-…